In [ ]:
import json
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import string


In [ ]:
nltk.download('stopwords')
stopwords_en = stopwords.words('english')
stopwords_fr = stopwords.words('french')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
with open("trial-v1.json", "r") as f:
    trial_data = json.load(f)

df = pd.json_normalize(trial_data)

#df.head(79)

# Splitting into MT/DM/PG

In [ ]:
mt_df = pd.DataFrame(columns=df.columns)
dm_df = pd.DataFrame(columns=df.columns)
pg_df = pd.DataFrame(columns=df.columns)


for index, row in df.iterrows():
    task = row['task']
    if task == 'MT':
        mt_df = mt_df.append(row)
    elif task == 'DM':
        dm_df = dm_df.append(row)
    elif task == 'PG':
        pg_df = pg_df.append(row)

mt_df.to_json(r'mt_oldindex_df.json', orient='records')
dm_df.to_json(r'dm_oldindex_df.json', orient='records')
pg_df.to_json(r'pg_oldindex_df.json', orient='records')

# Reset the index for each new DataFrame
mt_df.reset_index(drop=True, inplace=True)
dm_df.reset_index(drop=True, inplace=True)
pg_df.reset_index(drop=True, inplace=True)

<ipython-input-5-aa54e8066caf>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dm_df = dm_df.append(row)
<ipython-input-5-aa54e8066caf>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dm_df = dm_df.append(row)
<ipython-input-5-aa54e8066caf>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dm_df = dm_df.append(row)
<ipython-input-5-aa54e8066caf>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dm_df = dm_df.append(row)
<ipython-input-5-aa54e8066caf>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dm_df = dm_df.append(row)
<ipython-input-5-aa5

In [ ]:
mt_df.to_json(r'mt_df.json', orient='records')
dm_df.to_json(r'dm_df.json', orient='records')
pg_df.to_json(r'pg_df.json', orient='records')

# Lower-casing

In [ ]:
df['hyp'] = df['hyp'].str.lower()
df['src'] = df['src'].str.lower()
df['tgt'] = df['tgt'].str.lower()

# Punctuation removal

In [ ]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [ ]:
df['hyp'] = df['hyp'].apply(remove_punctuations)
df['src'] = df['src'].apply(remove_punctuations)
df['tgt'] = df['tgt'].apply(remove_punctuations)

# Stopwords Removal

In [ ]:
df['hyp'] = df['hyp'].apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stopwords_en))
df['tgt'] = df['tgt'].apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stopwords_en))
#df['src'] = df['src'].apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stopwords_en))

for i in range(0,80):
  if df.iloc[i]['task'] == 'MT':
    df['src'] = df['src'].apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stopwords_fr))
  else:
    df['src'] = df['src'].apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stopwords_en))

# Lemmatization (as extra feature)

In [ ]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
  return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

In [ ]:
df['hyp_lemmatized'] = df['hyp'].apply(lemmatize_text)
df['src_lemmatized'] = df['src'].apply(lemmatize_text)
df['tgt_lemmatized'] = df['tgt'].apply(lemmatize_text)

# Feature 'mutual words' added

In [ ]:
for i in range(0,80):
  hyp = w_tokenizer.tokenize(df.iloc[i]['hyp'])
  tgt = w_tokenizer.tokenize(df.iloc[i]['tgt'])

  mutual_elements = set(hyp) & set(tgt)
  mutual_words = len(mutual_elements)
  df.loc[i, 'mutual words'] = mutual_words

In [ ]:
mt_processed_df = pd.DataFrame(columns=df.columns)
dm_processed_df = pd.DataFrame(columns=df.columns)
pg_processed_df = pd.DataFrame(columns=df.columns)


for index, row in df.iterrows():
    task = row['task']
    if task == 'MT':
        mt_processed_df = mt_processed_df.append(row)
    elif task == 'DM':
        dm_processed_df = dm_processed_df.append(row)
    elif task == 'PG':
        pg_processed_df = pg_processed_df.append(row)


mt_processed_df.to_json(r'mt_processed_oldindex_df.json', orient='records')
dm_processed_df.to_json(r'dm_processed_oldindex_df.json', orient='records')
pg_processed_df.to_json(r'pg_processed_oldindex_df.json', orient='records')

# Reset the index for each new DataFrame
mt_processed_df.reset_index(drop=True, inplace=True)
dm_processed_df.reset_index(drop=True, inplace=True)
pg_processed_df.reset_index(drop=True, inplace=True)

<ipython-input-14-2e3656ab4457>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dm_processed_df = dm_processed_df.append(row)
<ipython-input-14-2e3656ab4457>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dm_processed_df = dm_processed_df.append(row)
<ipython-input-14-2e3656ab4457>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dm_processed_df = dm_processed_df.append(row)
<ipython-input-14-2e3656ab4457>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dm_processed_df = dm_processed_df.append(row)
<ipython-input-14-2e3656ab4457>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future

# Export file

In [ ]:
df.to_json(r'processed_df.json', orient='records')

In [ ]:
mt_processed_df.to_json(r'mt_processed_df.json', orient='records')
dm_processed_df.to_json(r'dm_processed_df.json', orient='records')
pg_processed_df.to_json(r'pg_processed_df.json', orient='records')